### Capturing word correlation in input data

In [1]:
import numpy as np
onehots = {}
onehots['cat'] = np.array([1,0,0,0])
onehots['the'] = np.array([0,1,0,0])
onehots['dog'] = np.array([0,0,1,0])
onehots['sat'] = np.array([0,0,0,1])

sentence = ['the','cat','sat']
x = onehots[sentence[0]]+onehots[sentence[1]]+onehots[sentence[2]]

print("Sentence Encoding:" + str(x))

Sentence Encoding:[1 1 0 1]


### Predicitng movie reviews

In [2]:
import sys


In [3]:
# reading the reviews and labels into a list
f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

f = open('labels.txt')
raw_labels = f.readlines()
f.close()

In [4]:
# mapping each review as tokens
tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

In [5]:
"""
creating a set of vocab taking each word from tokens and if len > 0
we add it to the list of words
"""
vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

In [6]:
#enumerating each word to find assign index for each of the words

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

In [7]:
# Converting each sentence to index and then creating set of each token to avoid duplicates

input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

In [8]:
# Converting labels to 0 and 1s
target_dataset = list()
for labels in raw_labels:
    if labels == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

In [9]:
import numpy as np
np.random.seed(1)

In [10]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [11]:
alpha, iteration = (0.01,2)
hidden_size = 100

In [12]:
weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,1)) - 0.1

#### Training with 2 layer network

In [13]:
correct,total = (0,0)
for iter in range(iteration):
    for i in range(len(input_dataset)-1000):
        x,y = (input_dataset[i],target_dataset[i])
        layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
        
        layer_2_delta = layer_2 - y
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)
        
        weights_0_1[x] -= layer_1_delta * alpha
        weights_1_2 -= np.outer(layer_1,layer_2_delta)*alpha
        
        if(np.abs(layer_2_delta)<0.5):
            correct += 1
        total += 1
        if(i % 10 == 9):
            progress = str(i/float(len(input_dataset)))
            sys.stdout.write('\rIter:'+str(iter)\
                +' Progress:'+progress[2:4]\
                             +'.'+progress[4:6]\
                +'% Training Accuracy:'\
                + str(correct/float(total)) + '%')
    print()
correct,total = (0,0)
for i in range(len(input_dataset)-1000,len(input_dataset)):
    x = input_dataset[i]
    y = target_dataset[i]
    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0)) 
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
    correct += 1
    total += 1
print("Test Accuracy:" + str(correct / float(total)))

Iter:0 Progress:05.95% Training Accuracy:0.5469798657718121%

Iter:0 Progress:12.35% Training Accuracy:0.6818770226537216%

Iter:0 Progress:18.71% Training Accuracy:0.7222222222222222%

Iter:0 Progress:24.71% Training Accuracy:0.7477346278317152%

Iter:0 Progress:30.67% Training Accuracy:0.76870925684485%

...

Iter:1 Progress:75.39% Training Accuracy:0.8610268378063011%

Iter:1 Progress:81.31% Training Accuracy:0.8626212497180239%

Iter:1 Progress:87.35% Training Accuracy:0.8639397905759162%

Iter:1 Progress:93.31% Training Accuracy:0.8648214663004437%

Iter:1 Progress:95.99% Training Accuracy:0.8659375%
Test Accuracy:1.0


#### Comparing word embeddings

In [14]:
from collections import Counter
import math 

def similar(target='beautiful'):
    target_index = word2index[target]
    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weights_0_1[index] - (weights_0_1[target_index])
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))

    return scores.most_common(10)

In [15]:
print(similar('beautiful'))

[('beautiful', -0.0), ('entertaining', -0.7465565551463458), ('recommended', -0.779555108217311), ('freedom', -0.7808616525739756), ('plenty', -0.7811375592487844), ('haunting', -0.7839631582799089), ('deserves', -0.7874797074290336), ('outstanding', -0.7880263885923592), ('atmosphere', -0.7992486752172738), ('great', -0.8003973017055172)]


In [16]:
print(similar('terrible'))

[('terrible', -0.0), ('horrible', -0.7487723560249665), ('boring', -0.7624635945633019), ('disappointment', -0.7691802437004109), ('avoid', -0.783921611381947), ('fails', -0.789242933233881), ('annoying', -0.7980921858729391), ('poor', -0.8022217860556791), ('badly', -0.8234072099106997), ('laughable', -0.8248411889308389)]


#### Filling in the blank

In [17]:
import sys,random,math
from collections import Counter
import numpy as np

In [18]:
np.random.seed(1)
random.seed(1)
f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()


tokens = list(map(lambda x:(x.split(" ")),raw_reviews))
wordcnt = Counter()
for sent in tokens:
    for word in sent:
        wordcnt[word] -= 1
vocab = list(set(map(lambda x:x[0],wordcnt.most_common())))

word2index = {}

for i,word in enumerate(vocab):
    word2index[word]=i
    
concatenated = list()
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
            concatenated.append(word2index[word])
        except:
            ""
        input_dataset.append(sent_indices)
concatenated = np.array(concatenated)
random.shuffle(input_dataset)


In [ ]:
alpha, iterations = (0.05, 2)
hidden_size,window,negative = (50,2,5)

weights_0_1 = (np.random.rand(len(vocab),hidden_size) - 0.5) * 0.2
weights_1_2 = np.random.rand(len(vocab),hidden_size)*0

layer_2_target = np.zeros(negative+1)
layer_2_target[0] = 1

def similar(target='beautiful'):
  target_index = word2index[target]

  scores = Counter()
  for word,index in word2index.items():
    raw_difference = weights_0_1[index] - (weights_0_1[target_index])
    squared_difference = raw_difference * raw_difference
    scores[word] = -math.sqrt(sum(squared_difference))
  return scores.most_common(10)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

for rev_i,review in enumerate(input_dataset * iterations):
  for target_i in range(len(review)):

    
    target_samples = [review[target_i]]+list(concatenated\
    [(np.random.rand(negative)*len(concatenated)).astype('int').tolist()])

    left_context = review[max(0,target_i-window):target_i]
    right_context = review[target_i+1:min(len(review),target_i+window)]

    layer_1 = np.mean(weights_0_1[left_context+right_context],axis=0)
    layer_2 = sigmoid(layer_1.dot(weights_1_2[target_samples].T))
    layer_2_delta = layer_2 - layer_2_target
    layer_1_delta = layer_2_delta.dot(weights_1_2[target_samples])

    weights_0_1[left_context+right_context] -= layer_1_delta * alpha
    weights_1_2[target_samples] -= np.outer(layer_2_delta,layer_1)*alpha

  if(rev_i % 250 == 0):
    sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)
        *iterations)) + "   " + str(similar('terrible')))
  sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)
        *iterations)))
print(similar('terrible'))

Progress:0.010621949620595341  [('terrible', -0.0), ('horrible', -2.717205586638701), ('brilliant', -3.8221067017718173), ('bad', -3.919026886107076), ('good', -4.289102034256767), ('great', -4.365336364583526), ('horrendous', -4.365421339303056), ('wonderful', -4.381293924826662), ('stupid', -4.481175930290705), ('slipshod', -4.515244992376265)])]498)]]3)]41)]])])]])]738)]]